### digit-recognizer

In [1]:
from __future__ import division, print_function, absolute_import
import numpy as np
import csv
import operator

In [2]:
#定义函数，将字符串转换为整数
def ArrayToInt(array):
    Array = np.mat(array)
    rows, columns = np.shape(array)
    newArray = np.zeros((rows, columns))
    for i in range(rows):
        for j in range(columns):
            newArray[i, j] = np.int(array[i, j])
    return newArray

In [3]:
#归一化
def Normalizer(array):
    rows, columns = np.shape(array)
    for i in range(rows):
        for j in range(columns):
            if array[i, j] != 0 :
                array[i, j] = 1
    return array

In [11]:
#加载training数据
def LoadTrainData():
    train_data = []
    with open('/tmp/data_input/kaggle/Digit_Recognition/train.csv') as file:
        datas = csv.reader(file)
        for data in datas:
            train_data.append(data)
    train_data = np.array(train_data[1:])
    train_data = np.array(train_data)
    labels = train_data[:, 0]
    train_datas = train_data[:, 1:]
    
    results = Normalizer(ArrayToInt(labels), ArrayToInt(train_datas))
    return results

In [12]:
train_data = LoadTrainData()
train_data

ValueError: not enough values to unpack (expected 2, got 1)

In [5]:
#加载test数据
def LoadTestData():
    test_data = []
    with open('/tmp/data_input/kaggle/Digit_Recognition/test.csv') as file:
        datas = csv.reader(file)
        for data in datas:
            test_data.append(data)
    test_data.remove(test_data[0])
    test_data = np.array(test_data)
    
    results = Normalizer(ArrayToInt(test_data))
    return results

In [6]:
def LoadTestResult():
    test_labels = []
    with open('/tmp/data_input/kaggle/Digit_Recognition/ample_submission.csv') as file:
        datas = csv.reader(file)
        for data in datas:
            test_labels.append(data)
    test_labels.remove(test_labels[0])
    test_labels = np.array(test_labels)
    
    results = ArrayToInt(test_labels[:, 1])
    

In [7]:
#处理
def Classifier(intX, dataSet, labels, k):
    intX = np.mat(intX)
    dataSet = np.mat(dataSet)
    labels = np.mat(labels)
    dataSet_size = dataSet.shape[0]
    
    diffmat = np.tile(intX, (dataSet_size, 1)) - dataSet
    sqdiffmat = np.power(sqdiffmat, 2)
    sqDistances = np.sum(sqdiffmat, axis=1)
    distances = np.sqrt(sqDistances)
    sortedDistances = np.argsort(distances)
    
    classcount = {}
    
    for i in range(k):
        voteIlabel = labels[sortedDistances[i], 0]
        classcount[voteIlabel] = classcount.get(voteIlabel, 0) + 1
        
    sortclasscount = sorted(classcount.iteritems(), key=operator.itemgetter(1), reverse=True)
    return sortclasscount[0][0]
    

In [8]:
#保存处理结果
def SaveResults(results):
    data = []
    with open('/tmp/data_output/kaggle/Digit_Recognition/result.csv', 'wb') as file:
        writer = csv.writer(file)
        for result in results:
            data.append(result)
            writer.writerrow(data)
    

In [9]:
def HandWritingClassifier():
    train_data, train_labels = LoadTrainData()
    test_data = LoadTestData()
    test_label = LoadTestResult()
    rows, columns = np.shape(test_data)
    error_count = 0
    resultList = []
    for i in rows:
        classcifierresults = Classifier(test_data[i], train_data, train_labels.transpose(), 5)
        resultList.append(classcifierresults)
        print ("The classifier result is %s , the real result is %s" % classcifierresults, test_label[0, i])
        
        if(classcifierresults != test_label[0, i]):
            error_count =+ 1.0
            
    print("The total error is %s" % error_count)
    SaveResults(resultList)